In [315]:
import socket
import pandas as pd
import numpy as np

In [316]:
key = [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
chunk_size = 7

In [317]:
def mapping(csv_filename, chunk_size=7):
    '''

    '''
    decoding_map = {}
    df =pd.read_csv(csv_filename)
    df['number'] =  pd.to_numeric(df['number']).astype(int)
    df['number'] = df['number'].apply(lambda x: bin(x & 0x7F)[2:].zfill(7)) 
    for index, row in df.iterrows():                                
        decoding_map[row[0]] = row[1]
    return decoding_map

In [318]:
HOST = '127.0.0.1'  # Dirección local (localhost)
PORT = 65432        # Puerto para la comunicación

In [319]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print("Bob esperando conexión...")
    conn, addr = s.accept()
    with conn:
        print(f"Conectado desde {addr}")
        encrypted_message = conn.recv(1024).decode()
        print("Mensaje cifrado recibido:", encrypted_message)

Bob esperando conexión...
Conectado desde ('127.0.0.1', 33579)
Mensaje cifrado recibido: 1110111011010010111000000100100011100100111111010100100010010101000110010000101110110100100001010101000011001001101111100011001100001101011110000010100111110111101100111001001


In [320]:
encrypted_bits = [int(digit) for digit in encrypted_message]

In [321]:
key_length = len(key)
extended_key = [key[i % key_length] for i in range(len(encrypted_bits))]

In [322]:
key_message = np.zeros(len(encrypted_bits), dtype=int)
for i in range(len(encrypted_bits)):
    if extended_key[i] == 1:
        key_message[i] = encrypted_bits[i] ^ 1
    else:
        key_message[i] = encrypted_bits[i]


In [323]:
flat_decrypted_string = "".join(map(str, key_message))

list_of_binary_strings_7bit = []
for i in range(0, len(flat_decrypted_string), chunk_size):
    chunk = flat_decrypted_string[i : i + chunk_size]
    list_of_binary_strings_7bit.append(chunk)

In [324]:
decoding_map_from_csv = mapping("alpha.csv", chunk_size)

In [325]:
decoded_message = ""
for i in list_of_binary_strings_7bit:
    decoded_message = decoded_message + decoding_map_from_csv[i]

In [326]:
decoded_message

"Don't talk about the club"